Requirement: by Toon

1.) install dlib -- from here https://www.youtube.com/watch?v=AUJKdehF2ZA

2.) opencv has to be version 4.5.1.48

In [17]:
import dlib, cv2,os
import matplotlib.pyplot as plt
import numpy as np
from imutils.face_utils import FaceAligner

In [18]:
pose_predictor=dlib.shape_predictor('../FaceReco-master/FaceReco/shape_predictor_68_face_landmarks.dat')
fa = FaceAligner(pose_predictor)
face_encoder=dlib.face_recognition_model_v1('../FaceReco-master/FaceReco/dlib_face_recognition_resnet_model_v1.dat')
detector = dlib.get_frontal_face_detector()
modelFile = '../FaceReco-master/FaceReco/opencv_face_detector_uint8.pb'
configFile = '../FaceReco-master/FaceReco/opencv_face_detector.pbtxt'
net = cv2.dnn.readNetFromTensorflow(modelFile, configFile)

In [19]:
faces=[]
name=[]
trainpath = "train"

In [20]:
for im in os.listdir(trainpath):
    print(im)
    img = cv2.imread(os.path.join(trainpath,im))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    frameHeight = img.shape[0]
    frameWidth = img.shape[1]
    blob = cv2.dnn.blobFromImage(img, 1.0, (300, 300), [104, 117,   123], False, False)
    net.setInput(blob)
    detections = net.forward()
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.7:
            x1 = int(detections[0, 0, i, 3] * frameWidth)
            y1 = int(detections[0, 0, i, 4] * frameHeight)
            x2 = int(detections[0, 0, i, 5] * frameWidth)
            y2 = int(detections[0, 0, i, 6] * frameHeight)
            faceAligned =fa.align(img, gray,dlib.rectangle(x1,y1,x2,y2))
            landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
            face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
            faces.append(face_descriptor)
            name.append(im)

Bill Gates8_608.jpg
Chris Evans5_1235.jpg
Dwayne Johnson10_1558.jpg
elon musk11_1533.jpg
Emma Watson32_1991.jpg
jeff bezos0_2040.jpg
Johnny Depp20_1841.jpg
Keanu Reeves12_1965.jpg
Mark Zuckerberg5_2063.jpg
scarlett johansson15_3937.jpg


In [21]:
faces = np.array(faces)
name = np.array(name)
np.save('face_repr.npy', faces)
np.save('labels.npy', name)

In [22]:
faces = np.load("face_repr.npy")
name = np.load("labels.npy")
# image = cv2.imread("../Test pytorch/Dataset/images/pins_Chris Evans/Chris Evans24_1196.jpg")
# gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

In [23]:
# frameHeight = image.shape[0]
# frameWidth = image.shape[1]
# blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
# net.setInput(blob)
# detections = net.forward()

In [24]:
# scores=[]
# for i in range(detections.shape[2]):
#     confidence = detections[0, 0, i, 2]
#     if confidence > 0.7:
#         x1 = int(detections[0, 0, i, 3] * frameWidth)
#         y1 = int(detections[0, 0, i, 4] * frameHeight)
#         x2 = int(detections[0, 0, i, 5] * frameWidth)
#         y2 = int(detections[0, 0, i, 6] * frameHeight)
#         faceAligned = fa.align(image, gray,dlib.rectangle(x1,y1,x2,y2))
#         landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
#         face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
#         score = np.linalg.norm(faces - np.array(face_descriptor), axis=1)
#         scores.append(score)
#         imatches = np.argsort(score)
#         score = score[imatches]
#         print(name[imatches][:10].tolist(), score[:10].tolist())

In [25]:
def predict(path):
    result_name = []    
    for file in os.listdir(path):
        image = cv2.imread(path +'/'+file)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        frameHeight = image.shape[0]
        frameWidth = image.shape[1]
        blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123], False, False)
        net.setInput(blob)
        detections = net.forward()
        scores=[]
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:
                x1 = int(detections[0, 0, i, 3] * frameWidth)
                y1 = int(detections[0, 0, i, 4] * frameHeight)
                x2 = int(detections[0, 0, i, 5] * frameWidth)
                y2 = int(detections[0, 0, i, 6] * frameHeight)
                faceAligned = fa.align(image, gray,dlib.rectangle(x1,y1,x2,y2))
                landmark = pose_predictor(faceAligned,dlib.rectangle(0,0,faceAligned.shape[0],faceAligned.shape[1]))
                face_descriptor = face_encoder.compute_face_descriptor(faceAligned, landmark, num_jitters=2)
                score = np.linalg.norm(faces - np.array(face_descriptor), axis=1)
                scores.append(score)
                imatches = np.argsort(score)
                score = score[imatches]
                a = name[imatches][:10].tolist()
                b = score[:10].tolist()
                print(a,b)
                c = name[imatches][0].split(' ')
                result_name.append(c[0])
                print(result_name)
    return result_name

In [26]:
result_name = predict("test/")

['Bill Gates8_608.jpg', 'Johnny Depp20_1841.jpg', 'Mark Zuckerberg5_2063.jpg', 'elon musk11_1533.jpg', 'jeff bezos0_2040.jpg', 'Keanu Reeves12_1965.jpg', 'Chris Evans5_1235.jpg', 'Dwayne Johnson10_1558.jpg', 'Emma Watson32_1991.jpg', 'scarlett johansson15_3937.jpg'] [0.5507180701954372, 0.6170697383285885, 0.624428702382983, 0.6585810858938059, 0.698128583015758, 0.7046866963650871, 0.7125096844253731, 0.7491717828030046, 0.7940857462302762, 0.9328482381090388]
['Bill']
['Bill Gates8_608.jpg', 'Johnny Depp20_1841.jpg', 'Emma Watson32_1991.jpg', 'elon musk11_1533.jpg', 'Mark Zuckerberg5_2063.jpg', 'Chris Evans5_1235.jpg', 'Keanu Reeves12_1965.jpg', 'jeff bezos0_2040.jpg', 'scarlett johansson15_3937.jpg', 'Dwayne Johnson10_1558.jpg'] [0.5918749624891091, 0.6661180458834827, 0.6678816240683428, 0.6960086553133509, 0.6988035428974398, 0.7473513194229863, 0.7645593256125293, 0.7748717359488722, 0.7831678356024538, 0.8249138948862507]
['Bill', 'Bill']
['Bill Gates8_608.jpg', 'Dwayne Johnson1

In [27]:
# print(result_name)
# print(len(name_list))

In [28]:
name_list = os.listdir("test")
name_list_split = [i.split(' ') for i in name_list]
print(name_list_split[2][0])

Bill


In [29]:
correct = 0
all = len(name_list)
for i in range(len(name_list)):
    if result_name[i]==name_list_split[i][0]:
        correct = correct + 1

In [30]:
print("Accuracy: ",correct*100/all,"%")

Accuracy:  94.0 %
